In [102]:
# y = y_ref - y_pred
# エネルギー差を取得

import os.path

import pandas as pd
import numpy as np
import glob

from utils.constants.dir_path import DataDirPath
from utils.constants.constants import Constants
from descriptors.base_info import get_reindex_base

In [103]:
dir_path = '/Users/y1u0d2/Desktop/Lab/result/nnp-train/rdf-inform/20211117'


In [104]:
epoch = 11
epoch = str(epoch).zfill(2)
base_df = get_reindex_base()
df_test = pd.read_csv(f'{dir_path}/analyze/testpoints.0000{epoch}.out.csv')
df_train = pd.read_csv(f'{dir_path}/analyze/trainpoints.0000{epoch}.out.csv')
df_test_new = pd.merge(df_test,base_df,left_on='index',right_on='structure_idx')
df_train_new = pd.merge(df_train,base_df,left_on='index',right_on='structure_idx')
df_concat = pd.concat([df_test_new, df_train_new])

alpha-critobalite 1 1150
alpha-quartz 1151 2250
beta-quartz 2251 3050
beta-trydymite 3051 4300
coesite 4301 5500
Fdd2-beta-critobalite 5501 6850
hex-trydymite 6851 7850
stishovite 7851 8550


In [105]:
norm_log_path = f'{dir_path}/input.nn'
with open(norm_log_path) as f:
    l_strip = [line.strip() for line in f.readlines()]
    l_strip = list(filter(None, l_strip))
    mean_arr = [line for line in l_strip if 'mean_energy' in line]
    norm_arr = [line for line in l_strip if 'conv_energy' in line]
for mean, norm in zip(mean_arr, norm_arr):
    mean = float(mean.split(' ')[-1])
    norm = float(norm.split(' ')[-1])
print(mean, norm)

df_concat['Ediff'] =df_concat.natom * 1/norm * (df_concat.Eref - df_concat.Ennp)
# df_concat['Ediff'] = df_concat.E - df_concat.E_nnp_from_norm

df_concat.sort_values('index', inplace=True)

-21.061372010002167 112.2077288944109


In [106]:
input_data_path = f'{dir_path}/input.data'
index = 0
output_data = []
with open(input_data_path, 'r') as f:
   l_strip = [line.strip() for line in f.readlines()]
   for i, line in enumerate(l_strip):
       if 'energy' in  line.split(' '):
           energy = df_concat.loc[df_concat['index'] == index, 'Ediff'].item()
           line = f'energy {energy}'
           index += 1
       output_data.append(line)

In [109]:
with open(f'{dir_path}/angular/input.data', 'w') as f:
    f.write('\n'.join(output_data))